## Using Condition

In [1]:
from multiprocessing import Process, Condition, Manager
import time
import logging

def setup_logging():
    # Configure the logging format to include timestamps
    log_format = '%(asctime)s: %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_format, datefmt='%H:%M:%S')

def producer(condition: Condition, shared_data):
    logging.info("Producer is producing data...")
    with condition:
        time.sleep(2)  # Simulate data production time
        shared_data.append("Data produced by producer")
        logging.info("Producer notifying consumers...")
        condition.notify_all()  # Notify all waiting consumers

def consumer(condition: Condition, shared_data, id: int):
    logging.info(f"Consumer {id} waiting for data...")
    with condition:
        logging.info(f"Consumer {id} has consumed {shared_data[0]}")

def run():
    # Shared objects
    condition = Condition()
    shared_data = Manager().list()

    # Creating a producer and consumer processes
    producer_process = Process(target=producer, args=(condition, shared_data))
    consumer_processes = [Process(target=consumer, args=(condition, shared_data, i)) for i in range(2)]

    # Start all processes
    producer_process.start()
    for cp in consumer_processes:
        cp.start()

    # Wait for
    producer_process.join()
    for cp in consumer_processes:
        cp.join()

setup_logging()
run()

18:04:34: Producer is producing data...
18:04:34: Consumer 0 waiting for data...
18:04:34: Consumer 1 waiting for data...
18:04:36: Producer notifying consumers...
18:04:36: Consumer 0 has consumed Data produced by producer
18:04:36: Consumer 1 has consumed Data produced by producer
